# Minor Project - HateSpeech Detection in hindi

# Imports

In [1]:
import pandas as pd
import numpy as np
import re
import emoji
import string
import csv
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import datasets
from sklearn import metrics

# DataSet

In [2]:
data = pd.read_csv(r'G:/PPSU Academic/SEM  4/SSIT2930 Minor Project-II/DataSet.csv')
data.head()

,Unique ID,Post,Labels Set
0,1,मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्क...,"hate,offensive"
1,2,सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...,non-hostile
2,3,"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील...",non-hostile
3,4,@prabhav218 साले जेएनयू छाप कमिने लोग हिन्दुओं...,"defamation,offensive"
4,5,#unlock4guidelines - अनलॉक-4 के लिए गाइडलाइन्स...,non-hostile


In [3]:
data["Post"][0]

'मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्के राम भक्त है और कुछ बाबर के साले है \n\n🙏 जय श्री राम 🙏'

# Text cleaning

In [4]:
STOPWORDS = ["अंदर","अत","अदि","अप","अपना","अपनि","अपनी","अपने","अभि","अभी","आदि","आप",
             "इंहिं","इंहें","इंहों","इतयादि","इत्यादि","इन","इनका","इन्हीं","इन्हें","इन्हों","इस","इसका","इसकि","इसकी",
             "इसके","इसमें","इसि","इसी","इसे","उंहिं","उंहें","उंहों","उन","उनका","उनकि","उनकी","उनके",
             "उनको","उन्हीं","उन्हें","उन्हों","उस","उसके",'थी,',"उसि","उसी","उसे","एक","एवं","एस","एसे","ऐसे","ओर",
             "और","कइ","कई","कर","करता","करते","करना","करने","करें","कहते","कहा",'कहाँ',"का","काफि","काफ़ी",
             "कि","किंहें","किंहों","कितना","किन्हें","किन्हों","किया","किर",'हम',"किस","किसि","किसी","किसे","की",
             "कुछ","कुल","के","को","कोइ","कोई","कोन","कोनसा","कौन","कौनसा","गया","घर","जब","जहाँ",
             "जहां","जा","जिंहें","जिंहों","जितना","जिधर","जिन","जिन्हें","जिन्हों","जिस","जिसे","जीधर","जेसा","जेसे",
             "जैसा","जैसे","जो","तक","तब","तरह","तिंहें","तिंहों","तिन","तिन्हें","तिन्हों","तिस","तिसे","तो","था","थि",
             "थी","थे","दबारा","दवारा","दिया","दुसरा","दुसरे","दूसरे","दो","द्वारा","न","नहिं","नहीं","ना","निचे",
             "निहायत","नीचे","ने","पर","पहले","पुरा","पूरा","पे","फिर","बनि","बनी","बहि","बही","बहुत","बाद","बाला",
             "बिलकुल","भि","भितर","भी","भीतर","मगर","मानो","मे","में","यदि","यह","यहाँ","यहां","यहि",'सैन्\u200dय',
             'सम्\u200dमान', 'नई', 'दिल्\u200dली', 'संस्\u200dकार।',
             "यही","या","यिह","ये","रखें","रवासा","रहा","रहे","ऱ्वासा","लिए","लिये","लेकिन","व","वगेरह","वरग","वर्ग",
             "वह","वहाँ","वहां","वहिं","वहीं","वाले","वुह","वे","वग़ैरह","संग","सकता","सकते","सबसे","सभि",'श्री','स्\u200dटीफन','माध्\u200dयम', 
             "सभी","साथ","साबुत","साभ","सारा","से","सो","हि","ही","हुअ","हुआ","हुइ","हुई","हुए","हे","हें","है",
             "हैं","हो","होता","होति","होती","होते","होना","होने"'थी,', 'वो','आ','हैं.', 'नए', 'हैं,','जाएगी', 'है।','है', 'क्यों','है,']


In [5]:
def stopword_removal(new_post):
    tokens = word_tokenize(new_post)
    stopword_remove_row = [word for word in tokens if word not in STOPWORDS]
    return stopword_remove_row

In [6]:
def emojis_removal(post):
    list1 = []
    for i in post:
        allchars = [str for str in i]
        emoji_list = [c for c in allchars if c in emoji.UNICODE_EMOJI]
        clean_text = ' '.join([str for str in i.split() if not any(i in str for i in emoji_list)])
        list1.append(clean_text)
    return list1

In [7]:
def puncuation_removal(list):
    list2 = []
    for i in list:
        new_str = re.sub(r'[a-zA-Z0-9\n\',.#@_:…।?/|!$*-]', r'',i)
        list2.append(new_str)
    return list2

In [8]:
post = data["Post"]
new_post = emojis_removal(post)
post1 = puncuation_removal(new_post)


In [9]:
post1

['मेरे देश के हिन्दु बहुत निराले है कुछ तो पक्के राम भक्त है और कुछ बाबर के साले है जय श्री राम',
 'सरकार हमेशा से किसानों की कमाई को बढ़ाने के लिए नईनई स्कीमें लाती रहती है ताकि उन पर ज्यादा आर्थिक बोझ न पड़े ',
 'सुशांत ने जो बिजनेस डील  जून को की थी वो डील दीपेश को सुशांत की हत्या के दिन ही क्यों याद आई देखिए पूछता है भारत अर्नब के साथ रिपब्लिक भारत पर    ',
 ' साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते है की संविधान सबको बराबर अधिकार देता है सच्चाई यह है कि यह बराबर अधिकार नहीं देता है',
 '  अनलॉक के लिए गाइडलाइन्स जारी   सितंबर से देशभर में मेट्रो सेवा शुरु होगी   सितंबर के बाद रैलियों और बाकी फंक्शन में  लोगों को इजाजत  कंटेनमेंट जोन में कोई छूट नहीं  सिनेमाहॉल अभी बंद रहेंगे   से वीं के छात्र  सितंबर के बाद स्कूल जा सकेंगे ',
 'चीन ने  में तर्क दिया की भारत का विपक्ष ही अजर\u200c मसुद को आतंकी नहीं मानता तो हम कैसे माने चुल्लू भर मूत्र में डूब मरो गद्दारों अब यह भारत के लोगों को सोचना है कि वो विपक्ष को वोट क्यों करेंविक्रम शर्मा',
 'देश में  के रिकॉर्ड मामले   ',
 '   से निक

In [10]:
data_x = post1
data_y = data["Labels Set"]


In [11]:
len(data_x)

5728

In [12]:
len(data_y)

5728

In [13]:
data_dict = { 
    
    "post": data_x,
    "labels": data_y,   
}


# Cleaned DataSet

In [14]:
data_new = pd.DataFrame(data_dict)
data_new.head()

,post,labels
0,मेरे देश के हिन्दु बहुत निराले है कुछ तो पक्के...,"hate,offensive"
1,सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...,non-hostile
2,सुशांत ने जो बिजनेस डील जून को की थी वो डील द...,non-hostile
3,साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते...,"defamation,offensive"
4,अनलॉक के लिए गाइडलाइन्स जारी सितंबर से देश...,non-hostile


In [15]:
l = []
for i in data_new["labels"]:
    a = i.split(",")
    for k in a:
        l.append(k)

        
print("hate", l.count("hate"))
print("offensive", l.count("offensive"))
print("non-hostile", l.count("non-hostile"))
print("defamation", l.count("defamation"))
print("fake", l.count("fake"))

hate 792
offensive 742
non-hostile 3050
defamation 564
fake 1144


In [16]:
df_dict = {
    "post": data_new["post"],
    "hate":np.zeros(5728),
    "offensive":np.zeros(5728),
    "non-hostile":np.zeros(5728),
    "defamation":np.zeros(5728),
    "fake":np.zeros(5728)
}



In [17]:
df_new = pd.DataFrame(df_dict)

In [18]:


for i in range(0,len(data_new["labels"])):
    a = data_new["labels"][i].split(",")
    for k in a:
        df_new[k][i] = 1

C:\Users\Dhruv.DHRUVPC\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


KeyboardInterrupt: 

In [ ]:
df_new.head()

#  CountVectorizer

In [ ]:
cv = CountVectorizer()

In [ ]:
x=cv.fit(df_new['post'])
#y=cv.fit(data_new['labels'])
print(x.vocabulary_)
#print(cv.get_feature_names())
x=cv.transform(df_new['post'])
#print(x.shape)


In [ ]:
target_data = []


for i in data_new['labels']:
    if('non-hostile' in i):
        target_data.append(int(1))
    else:
        target_data.append(int(0))

target_data

In [ ]:
x=cv.transform(data_new['post'])
print(x.shape)
print(len(target_data))

In [ ]:
dataset=x.toarray()
print(dataset)

# Tfidf

In [ ]:
tfidf = TfidfVectorizer()

In [ ]:
tfid_data = tfidf.fit_transform(data_new['post'])
print(tfid_data.shape)

In [ ]:
df = pd.DataFrame(tfid_data.toarray(),columns=cv.get_feature_names())
df.head()

# Split DataSet into train and test

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(vectorized,target_data,test_size=0.25, random_state=42)

In [ ]:
len(X_train)

In [ ]:
len(X_test)

In [ ]:
len(Y_train)

In [ ]:
len(Y_test)

In [ ]:
set_target_data=target_data[0:4296]
len(set_target_data)

In [ ]:
X_train

# Modal traning using Gaussian Naive Bayes

In [ ]:
model = GaussianNB()
model.fit(X_train,set_target_data)

In [ ]:
expected= set_target_data
predicated =model.predict(X_train)

In [ ]:
print(metrics.classification_report(expected,predicated))


In [ ]:
print(metrics.confusion_matrix(expected,predicated))